# Preview Support Tickets

In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import streamlit as st
import ast

session = get_active_session()

In [ ]:
use database TELCO_SUPPORT_DB;
use schema SUPPORT_DATA;

In [ ]:
df_support_tickets = session.table('SUPPORT_TICKETS')
df_support_tickets

## Custom email or text reponse generation for support tickets using LLMs

In [ ]:
prompt = """You are a customer support representative at a telecommunications company. 
Suddenly there is a spike in customer support tickets. 
You need to understand and analyze the support requests from customers.
Based on the root cause of the main issue in the support request, craft a response to resolve the customer issue.
Write a text message under 25 words, if the contact_preference field is text message.
Write an email in maximum of 100 words if the contact_preference field is email. 
Focus on alleviating the customer issue and improving customer satisfaction in your response.
Strictly follow the word count limit for the response. 
Write only email or text message response based on the contact_preference for every customer. 
Do not generate both email and text message response.
"""


prompt1 = """
Please write an email or text promoting a new plan that will save customers total costs. 
Also resolve the customer issue based on the ticket category. 
If the contact_preference is text message, write text message response in less than 25 words. 
If the contact_preference is email, write email response in maximum 100 words.
Write only email or text message response based on the contact_preference for every customer.
"""

In [ ]:
ticket_categories = ['Roaming fees', 'Slow data speed', 'Lost phone', 'Add new line', 'Closing account']

## Mistral-7b response

In [ ]:
df_mistral_7b_response_sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('mistral-7b',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as mistral_7b_response
from SUPPORT_TICKETS"""

df_mistral_7b_response = session.sql(df_mistral_7b_response_sql)
df_mistral_7b_response

## mistral-large response

In [ ]:
mistral_large_response_sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('mistral-large',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as mistral_large_response
from SUPPORT_TICKETS"""

df_mistral_large_response = session.sql(mistral_large_response_sql)
df_mistral_large_response

# Generate dataset to fine-tune mistral-7b

In [ ]:
# Stage 1: Filter by CONTACT_PREFERENCE
df_text = df_mistral_large_response.filter(F.col("CONTACT_PREFERENCE") == 'Text Message')
df_email = df_mistral_large_response.filter(F.col("CONTACT_PREFERENCE") == "Email")

# Stage 2: Apply word count logic
df_text_filtered = df_text.filter(F.regexp_count(F.col("MISTRAL_LARGE_RESPONSE"), r" ") <= 25)
df_email_filtered = df_email.filter(F.regexp_count(F.col("MISTRAL_LARGE_RESPONSE"), r" ") > 30)

# Combine the results
df_filtered = df_text_filtered.union(df_email_filtered)

df_filtered

In [ ]:
df_filtered.write.save_as_table("SUPPORT_TICKET_RESPONSES", mode="overwrite")  # "overwrite" can be changed to "append"

# Optional: Show the saved table to verify
saved_df = session.table("SUPPORT_TICKET_RESPONSES")
saved_df

In [ ]:
df_fine_tune = saved_df.with_column("prompt", 
                                    F.concat(F.lit(prompt), 
                                             F.lit(" "), 
                                             F.lit("Contact preference: "),
                                             F.col("contact_preference"), 
                                             F.lit(" "),
                                             F.col("ticket_category")))\
                        .select("ticket_id",
                                "prompt",
                                "mistral_large_response")

df_fine_tune.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style')

# Fine-tune mistral-7b

## Split data into training and evaluation

In [ ]:
train_df, eval_df = session.table("support_tickets_finetune_message_style").random_split(weights=[0.8, 0.2], seed=42)

train_df.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style_train')
eval_df.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style_eval')

In [ ]:
session.table('support_tickets_finetune_message_style_train').show(1)

In [ ]:
session.table('support_tickets_finetune_message_style_eval').show(1)

## *Fine-tune mistral-7b model using Cortex*

In [ ]:
select snowflake.cortex.finetune('CREATE', 
'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B', 
'mistral-7b', 
'SELECT prompt, mistral_large_response as completion from support_tickets_finetune_message_style_train',
'SELECT prompt, mistral_large_response as completion from support_tickets_finetune_message_style_eval');

In [ ]:
select snowflake.cortex.finetune('SHOW');

### See status of the fine tuning job

In [ ]:
select snowflake.cortex.finetune('DESCRIBE', 'ft_2a4d9d51-5969-4d9b-bde5-93bdc5414bb9');

# Inference using fine-tuned model

In [ ]:
fine_tuned_model_name = 'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B'

sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as fine_tuned_mistral_7b_model_response
from SUPPORT_TICKETS"""

df_fine_tuned_mistral_7b_response = session.sql(sql)
df_fine_tuned_mistral_7b_response

## Streamlit application to auto-generate custom emails and text messages

In [ ]:
st.subheader("Auto-generate custom emails or text messages")

with st.container():
    with st.expander("Enter customer request and select LLM", expanded=True):
        customer_request = st.text_area('Request',"""I traveled to Japan for two weeks and kept my data usage to a minimum. However, I was charged $90 in international fees. These charges were not communicated to me, and I request a detailed breakdown and a refund. Thank you for your prompt assistance.""")
    
        with st.container():
            left_col, right_col = st.columns(2)
            with left_col:
                selected_preference = st.selectbox('Select contact preference', ('Text message', 'Email'))
            with right_col:
                selected_llm = st.selectbox('Select LLM',('llama3-8b', 'mistral-7b', 'mistral-large', 'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B',))

with st.container():
    _,mid_col,_ = st.columns([.4,.3,.3])
    with mid_col:
        generate_template = st.button('Generate messages ⚡',type="primary")

with st.container():
    if generate_template:
        category_sql = f"""
        select snowflake.cortex.classify_text('{customer_request}', {ticket_categories}) as ticket_category
        """
        df_category = session.sql(category_sql).to_pandas().iloc[0]['TICKET_CATEGORY']
        df_category_dict = ast.literal_eval(df_category)
        st.subheader("Ticket category")
        st.write(df_category_dict['label'])

        message_sql = f"""
        select snowflake.cortex.complete('{selected_llm}',concat('{prompt}', '{customer_request}', '{selected_preference}')) as custom_message
        """
        df_message = session.sql(message_sql).to_pandas().iloc[0]['CUSTOM_MESSAGE']
        st.subheader(selected_preference)
        st.write(df_message)